In [ ]:
import pandas as pd
from data_preprocessing import *
from train_model_helper import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
import mlflow
import mlflow.sklearn

In [ ]:
# prepare the data
df = pd.read_csv('hand_landmarks_data.csv')
df = preprocess_data(df)
X, y = df
le = LabelEncoder()
y = le.fit_transform(y)

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# MLflow setup
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("hand_gestures_experiment")

# Log the dataset
dataset = mlflow.data.from_pandas(df, source="hand_landmarks_data.csv", name="hand_gestures")